# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [01:17<00:00, 15.51s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Walmart Deals Week: Up to 70% off + free shipping w/ $35\nDetails: Walmart's Deals Week event is still on for another two days, where you can shop lots of amazing deals and all-time lows on big-brand items (including LEGO, Apple, Samsung, and more). Although the banner says up to 50% off, we found deeper discounts within. Shipping is $6.99 or free on orders over $35 (pickup is also available, depending on your location) Shop Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Walmart-Deals-Week-Up-to-70-off-free-shipping-w-35/21752224.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung Galaxy Watch Ultra 47mm LTE Smartwatch (2025) Pre-Order: $50 off + up to $250 off w/ trade-in + free shipping
Details: It's now $500 with no trade required, $50 under our mention from yesterday. Plus, if you have a trade, you'll save up to another $250 off. The watch is due to release on July 25.  Shop Now at Samsung
Features: 
URL: https://www.dealnews.com/

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The Unlocked Samsung Galaxy Z Flip7 512GB Android Smartphone features a distinctive foldable design that makes it compact yet functional. With this model, you get substantial storage space alongside a premium feel and advanced camera capabilities, perfect for capturing high-quality images on the go. The phone is set to launch on July 25, and it brings cutting-edge technology to your fingertips, allowing for a seamless mobile experience.', price=999.99, url='https://www.dealnews.com/Unlocked-Samsung-Galaxy-Z-Flip7-512-GB-Android-Smartphone-Pre-Order-120-off-up-to-600-off-w-trade-free-shipping/21752156.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Samsung Galaxy Watch Ultra 47mm LTE Smartwatch is a high-performance wearable, equipped with advanced health and fitness tracking features. Its robust 47mm frame houses an AMOLED display, providing vibrant visuals and easy readability. This smartwatch integrates seamless connectivity options and offers a variety of fitness modes, making it the ideal companion for both athletes and tech users alike. Scheduled for release on July 25, this model is priced at $500, ensuring a stylish and functional experience.', price=500.0, url='https://www.dealnews.com/Samsung-Galaxy-Watch-Ultra-47-mm-LTE-Smartwatch-2025-Pre-Order-50-off-up-to-250-off-w-trade-in-free-shipping/21752168.html?iref=rss-c142'), Deal(product_description='The Jackery 5000 Plus Power Station is a powerful and versatile energy solution, boasting a 5,040Wh capacity that is expandable up to 60kWh. It features a state-of-the-art LiFePO4 battery for enhanced safety and longevity. Wit